## 1. アプリデータのタイトル列クリーンアップと整形
- 表記ゆれ対策のためのタイトル前処理

### 1.1. ライブラリとファイルの読み込み

In [19]:
import pandas as pd
import re

# 入力ファイルと出力ファイルのパス
input_path = "gamei_apps_sales_summary_utf8bom_ver3.csv"
output_full_path = "full_data_with_cleaned_titles.csv"
output_titles_only_path = "cleaned_unique_titles.csv"

# 元データの読み込み
df = pd.read_csv(input_path)

### 1.2. タイトルクリーンアップ関数の定義

In [20]:
def clean_title(title):
    if pd.isna(title):
        return None
    # あらゆる括弧とその中の文字列を削除（半角＋全角）
    return re.sub(r'[（\(\[【［][^）\)\]】］]*[）\)\]】］]', '', str(title)).strip()

### 1.3. クリーンなタイトル列を追加

In [21]:
df["クリーンタイトル"] = df["タイトル"].map(clean_title)

### 1.4. クリーンタイトル付きの元データを保存

In [22]:
# 元データ＋クリーンタイトル列を含めて保存
df.to_csv(output_full_path, index=False, encoding="utf-8-sig")
print(f"元データにクリーンタイトル列を加えて保存：{output_full_path}")

元データにクリーンタイトル列を加えて保存：full_data_with_cleaned_titles.csv


### 1.5. クリーンタイトルのみ抽出して保存

In [23]:
# クリーンタイトルのユニーク値のみ抽出
unique_titles = df["クリーンタイトル"].dropna().unique()
cleaned_df = pd.DataFrame(unique_titles, columns=["タイトル"])

# タイトル一覧を保存（統合作業用）
cleaned_df.to_csv(output_titles_only_path, index=False, encoding="utf-8-sig")
print(f"クリーンなタイトル一覧を保存：{output_titles_only_path}（{len(cleaned_df)} 件）")

クリーンなタイトル一覧を保存：cleaned_unique_titles.csv（393 件）


### 補足：保存した2つのCSVの使い分け
- `full_data_with_cleaned_titles`.csv → 統合や元データ分析用
- `cleaned_unique_titles.csv` → 検索やスクレイピング入力用

## 2. アプリタイトルから発売日データをスクレイピング

### 2.1.　ライブラリのインポート

In [24]:
# --- 必要なライブラリのインポート ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import traceback

### 2.2. 入出力ファイル・Chrome起動設定

In [25]:
# --- ChromeDriverの起動 ---
driver = webdriver.Chrome()

# --- 入力CSVの読み込み（"タイトル"列が必要） ---
df = pd.read_csv("cleaned_unique_titles.csv")
df["発売日"] = ""
df["備考"] = ""
total = len(df)
not_found_titles = []

### 2.4. メイン処理ループ

In [26]:
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{total}] 検索中：{title}")

    try:
        # トップページから検索
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # 検索結果が表示されるのを待機
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        )

        links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        game_links = []

        for link in links:
            href = link.get_attribute("href")
            if href and "/games/" in href:
                game_links.append(href)

        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            not_found_titles.append(title)
            continue

        # 詳細ページ処理
        release_dates = []

        for url in game_links:
            if not url.startswith("http"):
                url = "https://www.4gamer.net" + url

            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.TAG_NAME, "th"))
            )

            th_list = driver.find_elements(By.TAG_NAME, "th")

            # 発売日取得
            for th in th_list:
                if "発売日" in th.text:
                    td_list = th.find_elements(By.XPATH, "./following-sibling::td")
                    if len(td_list) >= 2:
                        date_text = td_list[1].text.strip()
                        if date_text and date_text not in ["-", "："] and re.match(r"\d{4}/\d{2}/\d{2}", date_text):
                            parsed = datetime.strptime(date_text, "%Y/%m/%d")
                            release_dates.append((parsed, date_text))
                            break

            # 備考取得
            for th in th_list:
                if "備考" in th.text:
                    td_list = th.find_elements(By.XPATH, "./following-sibling::td")
                    if len(td_list) >= 2:
                        a_tags = td_list[1].find_elements(By.TAG_NAME, "a")
                        remarks = [a.text.strip() for a in a_tags if a.text.strip()]
                        if remarks:
                            df.at[idx, "備考"] = ", ".join(remarks)
                    break

        # 発売日決定
        if release_dates:
            oldest = min(release_dates, key=lambda x: x[0])[1]
            df.at[idx, "発売日"] = oldest
            print(f" → 最も古い発売日：{oldest}")
            if df.at[idx, "備考"]:
                print(f" → 備考：{df.at[idx, '備考']}")
        else:
            print(" → 発売日が見つかりません")
            not_found_titles.append(title)

    except TimeoutException:
        print(" → 検索結果が見つかりません（タイムアウト）")
        not_found_titles.append(title)
        continue

    except Exception:
        print(" → エラーが発生しました")
        traceback.print_exc()
        not_found_titles.append(title)
        continue

[1/393] 検索中：スーパーガンダムロワイヤル
 → 最も古い発売日：2015/10/16
 → 備考：アニメ/コミック, 無料
[2/393] 検索中：武器よさらば
 → 最も古い発売日：2017/03/30
 → 備考：無料, プレイ人数：1人
[3/393] 検索中：スーパーロボット大戦X-Ω
 → 最も古い発売日：2015/10/05
 → 備考：アニメ/コミック, ロボット, 戦争物, 無料
[4/393] 検索中：FINAL FANTASY GRANDMASTERS
 → 最も古い発売日：2015/10/01
 → 備考：日本, ファイナルファンタジー, 無料, ファンタジー
[5/393] 検索中：ドラゴンプロジェクト
 → 最も古い発売日：2016/06/03
 → 備考：プレイ人数：1～4人, 協力プレイ, ファンタジー, 無料, 日本
[6/393] 検索中：戦の海賊
 → 最も古い発売日：2015/08/27
 → 備考：海洋, 無料
[7/393] 検索中：誰ガ為のアルケミスト
 → 最も古い発売日：2016/01/28
 → 備考：ファンタジー, 戦略級, 基本プレイ無料
[8/393] 検索中：デッキヒーローズ-本格派戦略カードゲーム-
 → 検索結果が見つかりません（タイムアウト）
[9/393] 検索中：美少女戦士セーラームーン　セーラームーンドロップス
 → 最も古い発売日：2015/09/03
 → 備考：パズル, 原作モノ, アニメ/コミック
[10/393] 検索中：モンスターハンター エクスプロア
 → 最も古い発売日：2015/09/03
 → 備考：無料, プレイ人数：1～4人, 協力プレイ, 日本
[11/393] 検索中：夢色キャスト
 → 最も古い発売日：2015/09/29
 → 備考：乙女ゲーム, 無料, 恋愛, 女性向け, 逢坂良太, 花江夏樹, 豊永利行, 上村祐翔, 林 勇, 小野友樹, 畠中 祐, 古川 慎, 斉藤壮馬, 鈴村健一, 山下大輝, 平川大輔, イケメン
[12/393] 検索中：グリムノーツ
 → 最も古い発売日：2016/01/21
 → 備考：無料
[13/393] 検索中：DAME×PRINCE -ダメ王子たちとのドタバタ恋愛ADV
 → 最も古い発売日：2016/03/31
 

Traceback (most recent call last):
  File "C:\Users\zakky\AppData\Local\Temp\ipykernel_19964\866777665.py", line 54, in <module>
    parsed = datetime.strptime(date_text, "%Y/%m/%d")
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\_strptime.py", line 674, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
                                    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\_strptime.py", line 456, in _strptime
    raise ValueError("unconverted data remains: %s" %
                      data_string[found.end():])
ValueError: unconverted data remains: （海外版）


 → 最も古い発売日：2016/08/03
 → 備考：東洋風, 無料
[55/393] 検索中：マギアレコード 魔法少女まどかマギカ外伝
 → 検索結果が見つかりません（タイムアウト）
[56/393] 検索中：ファンタジーライフ オンライン
 → 最も古い発売日：2018/07/23
 → 備考：ほのぼの, 生活系, 無料
[57/393] 検索中：マジカルデイズ｜魔法パズル×青春ストーリー！
 → 検索結果が見つかりません（タイムアウト）
[58/393] 検索中：戦国幻武～本格軍勢バトル～
 → 最も古い発売日：2017/06/07
 → 備考：戦国時代, 無料
[59/393] 検索中：BRAVELY DEFAULT FAIRY'S EFFECT
 → 最も古い発売日：2017/03/23
 → 備考：ファンタジー, 無料, 日本
[60/393] 検索中：歌マクロス スマホDeカルチャー
 → 最も古い発売日：2017/08/03
 → 備考：アニメ/コミック, 原作モノ, 無料, プレイ人数：1人
[61/393] 検索中：にゅ～パズ松さん 新品卒業計画
 → 最も古い発売日：2017/09/28
 → 備考：アニメ/コミック, パズル, 原作モノ, 無料
[62/393] 検索中：ドラゴンクエストライバルズ エース
 → 最も古い発売日：2017/11/02
 → 備考：カード, 対戦プレイ, 基本プレイ無料, ドラゴンクエスト
[63/393] 検索中：PaniPani -パラレルニクスパンドラナイト-
 → 最も古い発売日：2017/09/28
 → 備考：ファンタジー, 無料
[64/393] 検索中：どうぶつの森 ポケットキャンプ
 → 最も古い発売日：2017/11/21
 → 備考：日本, ほのぼの
[65/393] 検索中：ららマジ
 → 最も古い発売日：2017/01/25
 → 備考：ほのぼの, ファンタジー, プレイ人数：1人, 無料, 育成, 学園物
[66/393] 検索中：グラフィティスマッシュ
 → 最も古い発売日：2017/10/25
 → 備考：無料, プレイ人数：1人, 協力プレイ, 対戦プレイ
[67/393] 検索中：ジャンプチ ヒーローズ
 → 最も古い発売日：2018/03/28
 → 備考：アニメ/コミック

### 2.5. 保存と終了処理

In [27]:
# --- 結果保存 ---
df.to_csv("4gamer_output.csv", index=False, encoding="utf-8-sig")

# --- WebDriverの終了 ---
driver.quit()

print("\n完了しました。取得できなかったタイトル数:", len(not_found_titles))


完了しました。取得できなかったタイトル数: 63
